In [ ]:
pip install oracledb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.8 MB/s eta 0:00:00


In [ ]:
import oracledb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import config


warnings.filterwarnings("ignore", category=UserWarning)

connection = oracledb.connect(user=config.user, password=config.password,
                              dsn=config.dsn)
cursor = connection.cursor()

In [ ]:
# create table stat_opis (
# parameter_id varchar(50) primary Key,
# max_atk number,
# min_atk number,
# mean_atk number,
# mediana_atk number,
# std_atk number,
# max_move number,
# min_move number,
# mean_move number,
# mediana_move number,
# std_move number,
# turn_avg number
# );

In [ ]:
df = pd.read_sql('select * from performances', connection)
df_sim = pd.read_sql('select * from simulations', connection)

In [ ]:
sql_command = "INSERT INTO stat_opis VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12)"

In [ ]:
cursor.execute('DELETE FROM stat_opis')
for x in df.PARAMETER_ID.unique():
  df_p = df[df.PARAMETER_ID==x]
  df_sim_p = df_sim[(df_sim['RED_PLAYER']==x) | (df_sim['BLUE_PLAYER']==x)]
  data = (
      x,
      int(df_p.ATK_CNT.max()),
      int(df_p.ATK_CNT.min()),
      df_p.ATK_CNT.mean(),
      int(df_p.ATK_CNT.median()),
      df_p.ATK_CNT.std(),
      int(df_p.MOVE_CNT.max()),
      int(df_p.MOVE_CNT.min()),
      df_p.MOVE_CNT.mean(),
      int(df_p.MOVE_CNT.median()),
      df_p.MOVE_CNT.std(),
      df_sim_p.TURN_CNT.mean()
  )
  cursor.execute(sql_command, data)
connection.commit()